In [86]:
import pandas as pd
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier 
import matplotlib.pyplot as plt

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

### Import and analyse datasets

In [133]:
dataset_tool_sofist_comma = pd.read_csv("../data/model/dataset_tool_sofist_comma.csv")
dataset_cogroo_sofist_comma = pd.read_csv("../data/model/dataset_cogroo_sofist_comma.csv")

In [132]:
dataset_tool_sofist_comma.head()

,essay,CASING,CLICHES,COMPOUNDING,CONFUSED_WORDS,CONTRACTIONS,FORMAL_SPEECH,GRAMMAR,MISC,MISSPELLING,...,comp1,palavras_por_texto,paragrafos_por_texto,palavras_no_paragrafo_1,palavras_no_ultimo_paragrafo,palavras_nos_intermediarios,palavras_unicas,quantidade_de_sentencas_longas,quantidade_de_paragrafos_com_sentenca_unica,error
0,0,0,0,1,0,0,0,0,0,0,...,1.0,199,4,30,54,115,99,0,0,1
1,1,1,0,0,1,0,1,1,0,0,...,1.0,350,4,74,104,172,147,1,0,4
2,2,0,0,0,0,1,1,2,0,0,...,0.5,372,3,70,86,216,133,2,1,3
3,3,0,0,0,0,0,0,0,0,0,...,1.0,165,3,61,31,73,101,2,3,1
4,4,0,0,0,3,0,1,1,0,0,...,1.0,184,4,41,31,112,99,0,2,4


In [134]:
dataset_cogroo_sofist_comma.head()

,essay,government:GOVERNMENT,probs:paronyms,punctuation:BEFORE_SENTENCES,repetition:DUPLICATED_TOKEN,xml:1,xml:10,xml:103,xml:104,xml:105,...,comp1,palavras_por_texto,paragrafos_por_texto,palavras_no_paragrafo_1,palavras_no_ultimo_paragrafo,palavras_nos_intermediarios,palavras_unicas,quantidade_de_sentencas_longas,quantidade_de_paragrafos_com_sentenca_unica,error
0,1,0,0,0,0,1,0,0,0,0,...,1.0,350,4,74,104,172,147,1,0,4
1,2,0,0,0,0,0,0,0,0,1,...,0.5,372,3,70,86,216,133,2,1,3
2,4,0,0,0,0,0,0,0,0,0,...,1.0,184,4,41,31,112,99,0,2,4
3,6,0,0,0,0,0,0,0,0,0,...,0.5,227,4,52,48,127,132,0,0,3
4,7,0,0,0,0,0,0,0,0,0,...,1.0,276,4,37,74,165,114,0,0,2


### Pre-processing

In [135]:
X1 = dataset_tool_sofist_comma.drop(columns=['comp1', 'essay'])
y1 = dataset_tool_sofist_comma['comp1'].astype(str)
train_X1, test_X1, train_y1, test_y1 = train_test_split(X1,y1, test_size=0.2, random_state=42)

In [136]:
X2 = dataset_cogroo_sofist_comma.drop(columns=['comp1', 'essay'])
y2 = dataset_cogroo_sofist_comma['comp1'].astype(str)
train_X2, test_X2, train_y2, test_y2 = train_test_split(X2,y2, test_size=0.2, random_state=42)

In [137]:
X3 =  dataset_tool_sofist_comma[dataset_tool_sofist_comma['comp1'] != '0.0'].drop(columns=['comp1', 'essay'])
y3 = dataset_tool_sofist_comma[dataset_tool_sofist_comma['comp1'] != '0.0']['comp1'].astype(str)
train_X3, test_X3, train_y3, test_y3 = train_test_split(X3,y3, test_size=0.2, random_state=42)

In [138]:
X4 =  dataset_cogroo_sofist_comma[dataset_cogroo_sofist_comma['comp1'] != '0.0'].drop(columns=['comp1', 'essay'])
y4 = dataset_cogroo_sofist_comma[dataset_cogroo_sofist_comma['comp1'] != '0.0']['comp1'].astype(str)
train_X4, test_X4, train_y4, test_y4 = train_test_split(X4,y4, test_size=0.2, random_state=42)

### Generating Models

In [139]:
def calculate_metrics(model, _x_train, _y_train, _x_test, _y_test):
    train_metric = model.score(_x_train, _y_train)
    test_metric = model.score(_x_test, _y_test) 
    _y_pred = model.predict(_x_test)
    classes = model.classes_
    metrics = [
        ["train", train_metric],
        ["test", test_metric],
        ["accuracy", accuracy_score(_y_test, _y_pred)],
        ["f1-macro", f1_score(_y_test, _y_pred, average='macro')],
        ["mean-squared-error", mean_squared_error(_y_test, _y_pred)]
    ]
    return metrics

In [153]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


def GetScaledModel(nameOfScaler, nameOfMulticlassPred=None):
    pipelines = []
    if nameOfScaler == 'standard':
        scaler = StandardScaler()
    elif nameOfScaler =='minmax':
        scaler = MinMaxScaler()

    if nameOfMulticlassPred == 'onevsone':
        multiclass = OneVsOneClassifier
    elif nameOfMulticlassPred == 'onevsrest':
        multiclass = OneVsRestClassifier
    
    if nameOfMulticlassPred:
        pipelines.append((nameOfScaler+'KNN' , Pipeline([('Scaler', scaler),('KNN' , multiclass(KNeighborsClassifier()))])))
        pipelines.append((nameOfScaler+'NB'  , Pipeline([('Scaler', scaler),('NB'  , multiclass(GaussianNB()))])))
        pipelines.append((nameOfScaler+'SVM' , Pipeline([('Scaler', scaler),('SVM' , multiclass(SVC()))])))
        pipelines.append((nameOfScaler+'AB'  , Pipeline([('Scaler', scaler),('AB'  , multiclass(AdaBoostClassifier()))])  ))
        pipelines.append((nameOfScaler+'GBM' , Pipeline([('Scaler', scaler),('GMB' , multiclass(GradientBoostingClassifier()))])))
        pipelines.append((nameOfScaler+'RF'  , Pipeline([('Scaler', scaler),('RF'  , multiclass(RandomForestClassifier()))])))
    else: 
        pipelines.append((nameOfScaler+'KNN' , Pipeline([('Scaler', scaler),('KNN' , KNeighborsClassifier())])))
        pipelines.append((nameOfScaler+'NB'  , Pipeline([('Scaler', scaler),('NB'  , GaussianNB())])))
        pipelines.append((nameOfScaler+'SVM' , Pipeline([('Scaler', scaler),('SVM' , SVC())])))
        pipelines.append((nameOfScaler+'AB'  , Pipeline([('Scaler', scaler),('AB'  , AdaBoostClassifier())])  ))
        pipelines.append((nameOfScaler+'GBM' , Pipeline([('Scaler', scaler),('GMB' , GradientBoostingClassifier())])))
        pipelines.append((nameOfScaler+'RF'  , Pipeline([('Scaler', scaler),('RF'  , RandomForestClassifier())])))
    return pipelines

In [158]:
def BasedLine2(X_train, y_train,models):
    # Test options and evaluation metric
    num_folds = 10
    scoring = 'accuracy'

    results = []
    names = []
    for name, model in models:
        kfold = StratifiedKFold(n_splits=num_folds)
        print(model)
        cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        
    return names, results

In [159]:
models = GetScaledModel('minmax', 'onevsone')
names,results = BasedLine2(train_X1, train_y1,models)
models

Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('KNN', OneVsOneClassifier(estimator=KNeighborsClassifier()))])
minmaxKNN: 0.384296 (0.020808)
Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('NB', OneVsOneClassifier(estimator=GaussianNB()))])
minmaxNB: 0.339994 (0.087723)
Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('SVM', OneVsOneClassifier(estimator=SVC()))])
minmaxSVM: 0.443169 (0.013683)
Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('AB', OneVsOneClassifier(estimator=AdaBoostClassifier()))])
minmaxAB: 0.423943 (0.035863)
Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('GMB',
                 OneVsOneClassifier(estimator=GradientBoostingClassifier()))])
minmaxGBM: 0.429202 (0.034955)
Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('RF', OneVsOneClassifier(estimator=RandomForestClassifier()))])
minmaxRF: 0.426877 (0.037175)


[('minmaxKNN',
  Pipeline(steps=[('Scaler', MinMaxScaler()),
                  ('KNN', OneVsOneClassifier(estimator=KNeighborsClassifier()))])),
 ('minmaxNB',
  Pipeline(steps=[('Scaler', MinMaxScaler()),
                  ('NB', OneVsOneClassifier(estimator=GaussianNB()))])),
 ('minmaxSVM',
  Pipeline(steps=[('Scaler', MinMaxScaler()),
                  ('SVM', OneVsOneClassifier(estimator=SVC()))])),
 ('minmaxAB',
  Pipeline(steps=[('Scaler', MinMaxScaler()),
                  ('AB', OneVsOneClassifier(estimator=AdaBoostClassifier()))])),
 ('minmaxGBM',
  Pipeline(steps=[('Scaler', MinMaxScaler()),
                  ('GMB',
                   OneVsOneClassifier(estimator=GradientBoostingClassifier()))])),
 ('minmaxRF',
  Pipeline(steps=[('Scaler', MinMaxScaler()),
                  ('RF', OneVsOneClassifier(estimator=RandomForestClassifier()))]))]

In [149]:
models = GetScaledModel('minmax', 'onevsone')
names,results = BasedLine2(train_X2, train_y2, models)

Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('KNN', OneVsOneClassifier(estimator=KNeighborsClassifier()))])
minmaxKNN: 0.334394 (0.039695)
Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('NB', OneVsOneClassifier(estimator=GaussianNB()))])
minmaxNB: 0.072174 (0.028666)
Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('SVM', OneVsOneClassifier(estimator=SVC()))])
minmaxSVM: 0.408148 (0.031327)
Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('AB', OneVsOneClassifier(estimator=AdaBoostClassifier()))])
minmaxAB: 0.401026 (0.043176)
Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('GMB',
                 OneVsOneClassifier(estimator=GradientBoostingClassifier()))])
minmaxGBM: 0.394600 (0.047415)
Pipeline(steps=[('Scaler', MinMaxScaler()),
                ('RF', OneVsOneClassifier(estimator=RandomForestClassifier()))])
minmaxRF: 0.387465 (0.051333)


In [162]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']}
one_class_clf = GridSearchCV(SVC(), param_grid, scoring="accuracy")
clf = OneVsOneClassifier(one_class_clf)